## Spam Email Classifier with KNN using TF-IDF scores

1.   Assignment must be implemented in Python 3 only.
2.   You are allowed to use libraries for data preprocessing (numpy, pandas, nltk etc) and for evaluation metrics, data visualization (matplotlib etc.).
3.   You will be evaluated not just on the overall performance of the model and also on the experimentation with hyper parameters, data prepossessing techniques etc.
4.   The report file must be a well documented jupyter notebook, explaining the experiments you have performed, evaluation metrics and corresponding code. The code must run and be able to reproduce the accuracies, figures/graphs etc.
5.   For all the questions, you must create a train-validation data split and test the hyperparameter tuning on the validation set. Your jupyter notebook must reflect the same.
6.   Strict plagiarism checking will be done. An F will be awarded for plagiarism.

**Task: Given an email, classify it as spam or ham**

Given input text file ("emails.txt") containing 5572 email messages, with each row having its corresponding label (spam/ham) attached to it.

This task also requires basic pre-processing of text (like removing stopwords, stemming/lemmatizing, replacing email_address with 'email-tag', etc..).

You are required to find the tf-idf scores for the given data and use them to perform KNN using Cosine Similarity.

# CODE

### Import necessary libraries

In [2]:
import os
import re
import sys
import math
import time
import string
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import nltk.corpus
import nltk
nltk.download('punkt')
nltk.download('stopwords')
lemmer=nltk.WordNetLemmatizer()
stop_words=nltk.corpus.stopwords.words('english')
from nltk.tokenize import RegexpTokenizer
tokenizer = RegexpTokenizer(r'\w+')

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer
countvectorizer = CountVectorizer(analyzer= 'word', stop_words='english')
tfidfvectorizer = TfidfVectorizer(analyzer='word',stop_words= 'english')

[nltk_data] Downloading package punkt to /home/archit/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /home/archit/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


### Load dataset

In [3]:
path = "./emails.txt"
data = pd.read_csv(path, sep='\t', header=None)
data.rename({0: 'label', 1:'mail_text'}, axis='columns', inplace=True)

### Preprocess data

In [4]:
def del_up(text):
    '''
        convert all uppercase letters to lowercase
        replace email address with 'email-tag'
        stop words removal
        lemming the words
    '''
    text = text.lower()
    if text in stop_words:
        return False, text
    regex = r'\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Z|a-z]{2,}\b'
    if(re.fullmatch(regex, text)):
        return True, 'email-tag'
    return True, lemmer.lemmatize(text)


In [5]:
def tok_pun(sentence):
    token_list=[]
    for i in tokenizer.tokenize(sentence):
        a,b=del_up(i)
        if a==True:
            token_list.append(b)
    # return " ".join(token_list)
    return token_list

def label_it(label):
    if label == 'ham':
        return 0
    return 1

In [6]:
data['tokenized_text'] = data['mail_text'].apply(lambda x: tok_pun(x))
data["label_val"] = data["label"].apply(lambda x: label_it(x))

In [8]:
def unique_words_in_data(data):
    data.head()
    columns=[]
    for x in data['tokenized_text']:
        for y in x:
            if y not in columns:
                columns.append(y)
    return columns

In [94]:
def idf(data,unique_words):
    idf_dict={}
    N=len(data)
    for i in unique_words:
        count=0
        for x in data['tokenized_text']:
            if i in x:
                count=count+1
        idf_dict[i]=(math.log((N)/(count)))
    return idf_dict 

def tfidf(data,idf_dic):
    tfidf_array=[]
    temp_arr={}
    for i in range(len(data)):
        x=data.loc[i]["tokenized_text"]
        temp_arr={}
        if len(x)==0:
            pass
            # print(i)
        else:
            for y in x:
                if y not in temp_arr:
                    temp_arr[y]=0
                temp_arr[y]+=1
            
            for y in temp_arr:
                temp_arr[y]=math.log(1+temp_arr[y])*idf_dic[y]
        tfidf_array.append(temp_arr)
    return tfidf_array


In [95]:
unique_words = unique_words_in_data(data)


In [96]:
idf_dic=idf(data,unique_words)


In [97]:
tfidf_array=tfidf(data,idf_dic)

### Split data

In [99]:
train_X, test_X, train_y, test_y = train_test_split(tfidf_array, data['label_val'], test_size=0.1, random_state=0)
train_X, val_X, train_y, val_y = train_test_split(train_X,train_y, test_size=1/9, random_state=0)

### Train your KNN model (reuse previously iplemented model built from scratch) and test on your data

***1. Experiment with different distance measures [Euclidean distance, Manhattan distance, Hamming Distance] and compare with the Cosine Similarity distance results.***

In [100]:
def moderator(dic1,dic2):
    dic3={}
    dic4={}
    for x in dic1:
        dic3[x]=dic1[x]
        dic4[x]=0.0
    for x in dic2:
        dic3[x]=0.0
        dic4[x]=dic2[x]
    dic3=np.array(list(dic3.values()))
    dic4=np.array(list(dic4.values()))
    return dic3,dic4

In [182]:
def euclidean_distance(X,Y):
    X,Y=moderator(X,Y)
    return np.sqrt(np.sum((X - Y)**2))

def manhattan_distance(X,Y):
    X,Y=moderator(X,Y)
    return np.sum(np.abs(X - Y))

def hamming_distance(X,Y):
    X,Y=moderator(X,Y)
    return np.sum(X != Y)

def cosine_similarity_distance(X,Y):
    X,Y=moderator(X,Y)
    return np.dot(X,Y) / (np.linalg.norm(X) * np.linalg.norm(Y))

In [181]:
def predict_label(email_X,train_X,train_y,k_value,distance_metric):
    spam_label=[]
    ham_label=[]
    arr1=email_X
    arr2=[]
    scores={}
    for idx,x in enumerate(train_X):
        arr2=x
        scores[idx]=distance_metric(arr1,arr2)
    if distance_metric==hamming_distance:
        sorted_scores=sorted(scores.items(), key=lambda x:x[1], reverse=False)[:k_value]
    else:
        sorted_scores=sorted(scores.items(), key=lambda x:x[1], reverse=True)[:k_value]
    for x in sorted_scores:
        if train_y[x[0]]==0:
            ham_label.append(x[0])
        else:
            spam_label.append(x[0])
    if len(ham_label)>len(spam_label):
        return 0
    elif len(ham_label)<len(spam_label):
        return 1
    else:
        if ham_label[0]<spam_label[0]:
            return 1
        elif ham_label[0]>spam_label[0]:
            return 0
        else:
            return random.choice([0,1])


In [148]:
def my_knn_predict(test_X,train_X,train_y,k_value,distance_metric=cosine_similarity_distance):
    '''
        distance matrix allowed are:
            euclidean_distance,
            hamming_distance,
            manhattan_distance,
            cosine_similarity_distance(default)
    '''
    predicted_label={}
    for idx,x in enumerate(test_X):
        if(idx%1000==0):
            print(idx,end="\r")
        predicted_label[idx]=predict_label(x,train_X,train_y,k_value,distance_metric)
    print("done!")
    return predicted_label

In [168]:
def my_knn_accuracy(predicted_label,test_y):
    count=0
    for x in predicted_label.keys():
        # print(x,predicted_label[x],test_y.iloc[x])
        if predicted_label[x]==test_y.iloc[x]:
            count+=1
    return count/len(predicted_label)

***2. Explain which distance measure works best and why? Explore the distance measures and weigh their pro and cons in different application settings.***

In [150]:
def try_diff_k_on_val(train_X,train_y,val_X,val_y,distance_metric,k=False,arr=[]):
    k_values=[]
    pre_k_acc=[]
    pre_k_acc_val=[]
    if k==True :
        k_values=arr
    else:
        for i in range(1,15,2):
            k_values.append(i)
    for i in k_values:
        temp=my_knn_predict(val_X,train_X,train_y,i,distance_metric)
        pre_k_acc_val.append(temp)
        pre_k_acc.append(my_knn_accuracy(temp,val_y))
    plt.plot(k_values,pre_k_acc_val)
    plt.show()
    return k_values,pre_k_acc,pre_k_acc_val

In [ ]:
ecu_a,ecu_b,ecu_c=try_diff_k_on_val(train_X,train_y,val_X,val_y,distance_metric=euclidean_distance)
man_a,man_b,man_c=try_diff_k_on_val(train_X,train_y,val_X,val_y,distance_metric=manhattan_distance)
ham_a,ham_b,ham_c=try_diff_k_on_val(train_X,train_y,val_X,val_y,distance_metric=hamming_distance)
cos_a,cos_b,cos_c=try_diff_k_on_val(train_X,train_y,val_X,val_y,distance_metric=cosine_similarity_distance)

In [170]:
euclidean_predict=my_knn_predict(test_X,train_X,train_y,5,euclidean_distance)
my_knn_accuracy(euclidean_predict,test_y)


done!


0.8620071684587813

In [171]:
manhattan_predict=my_knn_predict(test_X,train_X,train_y,5,manhattan_distance)
my_knn_accuracy(manhattan_predict,test_y)

done!


0.8620071684587813

In [183]:
hamming_predict=my_knn_predict(test_X,train_X,train_y,5,hamming_distance)


KeyError: 1771

In [178]:
print(hamming_distance)

<function hamming_distance at 0x7f5505992d30>


In [ ]:
my_knn_accuracy(hamming_predict,test_y)

In [173]:
cosine_similarity_predict=my_knn_predict(test_X,train_X,train_y,5,cosine_similarity_distance)
my_knn_accuracy(cosine_similarity_predict,test_y)

/tmp/ipykernel_14895/1863797271.py:15: RuntimeWarning: invalid value encountered in double_scalars
  return np.dot(X,Y) / (np.linalg.norm(X) * np.linalg.norm(Y))


KeyError: 2

***3. Report Mean Squared Error(MSE), Mean-Absolute-Error(MAE), R-squared (R2) score in a tabular form***

***4. Choose different K values (k=1,3,5,7,11,17,23,28) and experiment. Plot a graph showing R2 score vs k.***

### Train and test Sklearn's KNN classifier model on your data (use metric which gave best results on your experimentation with built-from-scratch model.)

***Compare both the models result.***

***What is the time complexity of training using KNN classifier?***

***What is the time complexity while testing? Is KNN a linear classifier or can it learn any boundary?***